# Titanic dataset Neural Network 

## Introduction
The purpose of this work is to apply a basic neural network (NN) to the Titanic dataset that is part of the popular Kaggle machine learning competition (https://www.kaggle.com/c/titanic/). This dataset consists of a set of two separate csv files 'train' and 'test' that contain 891 and 418 instances of 11 independent variables respectively. These variables describe properties about passengers on that fateful and iconic voyage across the Atlantic. In addition to the 11 independent variables, the train file contains a dependent variable 'Survived' that refers to whether a passenger on the titanic voyage survived (denoted by a 1) or not (denoted by a 0). The test file does not contain the dependent variable 'survived'. Each instance for both files represents a single passenger on the ship. The purpose of this exercise is to develop and train a machine learning model on the 'train' data, and then use this model (a Neural Network in this case) to predict whether the passengers described by the 'test' data 'survive' or not. 
This is then submitted to the Kaggle website for assessment. 

In [1]:
# First step is to import all relevant packages used in this work

import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from numpy.random import seed
from keras.models import Sequential # intitialize the ANN
from keras.layers import Dense      # Used to create 'Dense' layers into the NN
from keras.layers import Dropout    # Dropouts allow for node dropout as part of the model

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV


In [2]:
# The data was downloaded from the Kaggle website. To run this project, you will need to have these two csv
# available. 
# Import the data and store as dataframes 
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
# Making copies of each to modify for the NN while maintaining original versions.
train1 = train.copy()
test1 = test.copy()

## Data wrangling

In [4]:
# Neural Networks are only able to operate with numerical values. Looking at the data types for the train dataset,
# some of the variables are objects. 
train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

### First looking at Sex

In [5]:
# In order to be compatible with a NN, the data must be processed into the correct format. 
# Setting Sex as a category
train1['Sex'] = train1['Sex'].astype('category')

# Now converting the categories to coded values (either a 1 for male or a 0 for female)
train1['Sex'] = train1['Sex'].cat.codes


### Extracting title information from the 'Name' descriptor
The 'Name' variable contains a potentially useful value. While the name itself of the passenger is too random and unlikely to contribute to the survival of the passenger, their title may.

In [6]:
# Looking to extract title information from the Name descriptors
train1['Title'] = train1.Name.map( lambda x: x.split(',')[1].split( '.' )[0].strip())

# Now to get an over all count for all types of titles in the 'train' dataframe.
train1['Title'].value_counts()

Mr              517
Miss            182
Mrs             125
Master           40
Dr                7
Rev               6
Col               2
Mlle              2
Major             2
Jonkheer          1
Lady              1
Mme               1
Sir               1
Ms                1
the Countess      1
Capt              1
Don               1
Name: Title, dtype: int64

In [7]:
# Clearly there are a number of titles for which there are single examples. Many of these are somewhat
# Similar/redundant with other more populated examples.
# Looking to replace all equivalent titles with more common versions
# Replace Mlle with Miss.
train1['Title'] = train1['Title'].replace('Mlle', 'Miss')
# Replace Mme, Lady and Ms with Mrs.
train1['Title'] = train1['Title'].replace(['Mme','Lady','Ms'], 'Mrs')
# Generate a list of counts for all remaining titles.
title_counts = train1['Title'].value_counts()

In [8]:
# Now to bundle all rare titles into 'others'
# Creating a dataframe of titles and their counts to manipulate in a moment.
counts = pd.DataFrame({'Title':title_counts.index, 'Count':title_counts.values})
# A subset of counts that has titles with less than or equal to 20 instances.
rare_title = counts.loc[counts['Count'] <= 20]
# Getting a list of all rate titles.
rare_title_list = rare_title['Title'].tolist()

# Using the rate_title_list to replace rare titles with 'Others' in the train1 dataframe.
train1['Title'] = train1['Title'].replace(rare_title_list, 'Others')
# Now to get a final count of modified title variable.
train1['Title'].value_counts()

Mr        517
Miss      184
Mrs       128
Master     40
Others     22
Name: Title, dtype: int64

### Binarisation of 'Embarked' and 'Title'
Here I apply a one-hot encoding (or binarisation) of the Embarked and Title data. This is ultimately a process where for each of these two independent variables, a new varialbe is created for each category within either Embarked or Title. For example, for Embarked, there are three categories, C, Q, and S. So now there will be three new independent variables, Embarked_C, Embarked_Q, and Embarked_S, and the original Embarked is removed. Now each new variable is a list of 1's and 0's (hense binarized), and any instance for which a passenger belonged to this category of Embarked receives a 1. Otherwise it is gets a 0. This more numerical representation of the same data is compatible with a Neural Network.

In [9]:
# Again in an effort to make all independent variables numerics, here I will
# subset all other categorical variables that need to be encoded (binarised). 
categorical = ['Embarked', 'Title']

for var in categorical:
    train1 = pd.concat([train1, pd.get_dummies(train1[var], prefix=var)], axis = 1)
    del train1[var]

# Now to remove not only the original Embarked and Title, but all unused variables in this NN.
train1.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

train1

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Others
0,0,3,1,22.0,1,0,7.2500,0,0,1,0,0,1,0,0
1,1,1,0,38.0,1,0,71.2833,1,0,0,0,0,0,1,0
2,1,3,0,26.0,0,0,7.9250,0,0,1,0,1,0,0,0
3,1,1,0,35.0,1,0,53.1000,0,0,1,0,0,0,1,0
4,0,3,1,35.0,0,0,8.0500,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,0,0,13.0000,0,0,1,0,0,0,0,1
887,1,1,0,19.0,0,0,30.0000,0,0,1,0,1,0,0,0
888,0,3,0,NaN,1,2,23.4500,0,0,1,0,1,0,0,0
889,1,1,1,26.0,0,0,30.0000,1,0,0,0,0,1,0,0


### Scaling continuous data
A quick survey of the data will reveal that while some parameters are binary, others vary as floats with much larger values ranges. This can be problematic for a NN to maintain sensitivity across the nodes. So we apply a scaleing procedure to all variables. 

In [10]:
# Now I need to scale all the continuous variables
continuous = ['Age', 'Fare', 'Parch', 'Pclass', 'SibSp']

for var in continuous:
    train1[var] = train1[var].astype('float64')
    train1[var] = StandardScaler().fit_transform(train1[var].values.reshape(-1, 1))

# This makes all of these 'continuous variables to have the means substacted and to vary around zero by the variance
# of the data.'
train1.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Others
count,891.000000,8.910000e+02,891.000000,7.140000e+02,8.910000e+02,8.910000e+02,8.910000e+02,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,-8.772133e-17,0.647587,2.388379e-16,4.386066e-17,5.382900e-17,3.987333e-18,0.188552,0.086420,0.722783,0.044893,0.206510,0.580247,0.143659,0.024691
std,0.486592,1.000562e+00,0.477990,1.000701e+00,1.000562e+00,1.000562e+00,1.000562e+00,0.391372,0.281141,0.447876,0.207186,0.405028,0.493796,0.350940,0.155270
min,0.000000,-1.566107e+00,0.000000,-2.016979e+00,-4.745452e-01,-4.736736e-01,-6.484217e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,-3.693648e-01,0.000000,-6.595416e-01,-4.745452e-01,-4.736736e-01,-4.891482e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,8.273772e-01,1.000000,-1.170488e-01,-4.745452e-01,-4.736736e-01,-3.573909e-01,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,1.000000,8.273772e-01,1.000000,5.718310e-01,4.327934e-01,-4.736736e-01,-2.424635e-02,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,1.000000,8.273772e-01,1.000000,3.465126e+00,6.784163e+00,6.974147e+00,9.667167e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Dealing with NaN entries
With the data almost being ready, a quick check for the presence of empty values or NaN's.

In [11]:
train1.isnull().sum()

Survived          0
Pclass            0
Sex               0
Age             177
SibSp             0
Parch             0
Fare              0
Embarked_C        0
Embarked_Q        0
Embarked_S        0
Title_Master      0
Title_Miss        0
Title_Mr          0
Title_Mrs         0
Title_Others      0
dtype: int64

In [12]:
# Clearly there are a significant number of NaN entries for the Age variable. 
# Replace all NaN's in Age with mean of all age values. 

train1['Age'] = train1['Age'].fillna(train1['Age'].mean())




### Train1 data is now ready
The dataframe now contains 15 columns of variables, including the Survived dependent variable.

In [13]:
train1

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Others
0,0,0.827377,1,-5.303766e-01,0.432793,-0.473674,-0.502445,0,0,1,0,0,1,0,0
1,1,-1.566107,0,5.718310e-01,0.432793,-0.473674,0.786845,1,0,0,0,0,0,1,0
2,1,0.827377,0,-2.548247e-01,-0.474545,-0.473674,-0.488854,0,0,1,0,1,0,0,0
3,1,-1.566107,0,3.651671e-01,0.432793,-0.473674,0.420730,0,0,1,0,0,0,1,0
4,0,0.827377,1,3.651671e-01,-0.474545,-0.473674,-0.486337,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,-0.369365,1,-1.859368e-01,-0.474545,-0.473674,-0.386671,0,0,1,0,0,0,0,1
887,1,-1.566107,0,-7.370406e-01,-0.474545,-0.473674,-0.044381,0,0,1,0,1,0,0,0
888,0,0.827377,0,2.388379e-16,0.432793,2.008933,-0.176263,0,0,1,0,1,0,0,0
889,1,-1.566107,1,-2.548247e-01,-0.474545,-0.473674,-0.044381,1,0,0,0,0,1,0,0


### Application of the same modifications to the 'Test1' dataframe

In [14]:
# Now applying these steps to test1 as well

# First the gender transformation.
test1['Sex'] = test1['Sex'].astype('category')
test1['Sex'] = test1['Sex'].cat.codes

# Now to split out the titles.
test1['Title'] = test1.Name.map( lambda x: x.split(',')[1].split( '.' )[0].strip())
# Looking to replace all equivalent titles with more common versions.
test1['Title'] = test1['Title'].replace('Mlle', 'Miss')
test1['Title'] = test1['Title'].replace(['Mme','Lady','Ms'], 'Mrs')
title_counts2 = test1['Title'].value_counts()

# Now to bundle all rare titles into 'others'
counts2 = pd.DataFrame({'Title':title_counts2.index, 'Count':title_counts2.values})

rare_title2 = counts2.loc[counts2['Count'] <= 20]
rare_title_list2 = rare_title2['Title'].tolist()
test1['Title'] = test1['Title'].replace(rare_title_list2, 'Others')

# Removing the variables that will not be used in this analysis
test1.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

for var in categorical:
    test1 = pd.concat([test1, pd.get_dummies(test1[var], prefix=var)], axis = 1)
    del test1[var]

# Now I need to scale all the continuous variables
for var in continuous:
    test1[var] = test1[var].astype('float64')
    test1[var] = StandardScaler().fit_transform(test1[var].values.reshape(-1, 1))

test1['Age'] = test1['Age'].fillna(train1['Age'].mean())
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [15]:
# Having a look for Null values.
test1.isnull().sum()


Pclass          0
Sex             0
Age             0
SibSp           0
Parch           0
Fare            1
Embarked_C      0
Embarked_Q      0
Embarked_S      0
Title_Master    0
Title_Miss      0
Title_Mr        0
Title_Mrs       0
Title_Others    0
dtype: int64

In [16]:
# One was found for Fare.
test1['Fare'] = test1['Fare'].fillna(train1['Fare'].mean())

### The Test1 dataframe is ready to be applied to a NN model

In [17]:
# In order to first generate and train a model, the train1 dataset must be divided into independent and dependent 
# variables.
X_train = train1[pd.notnull(train1['Survived'])].drop(['Survived'], axis=1)
Y_train = train1[pd.notnull(train1['Survived'])]['Survived']
Y_train

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

## Development of a Neural Network
Hyperparameters are variables that determine the structure of the NN (eg. hidden layers), as well as the parameters that determine how the NN is trained (eg. learning rate).

In [18]:
# This is a function that will allow for the parameterisation of the hyperparameters in the NN.

def create_model(lyrs=[8], act='relu', opt='Adam', dr=0.0):
    # Setting random seed
    seed(42)
    tf.random.set_seed(42)
    
    model = Sequential()
    
    # Create the first layer
    model.add(Dense(lyrs[0], input_dim=X_train.shape[1], activation=act))
    
    # Create additional hidden layer
    for i in range(1, len(lyrs)):
        model.add(Dense(lyrs[i], activation=act))
    
    # Add dropout, default = none
    model.add(Dropout(dr))
    
    # Create output layer
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model


In [19]:
# Now to create the first model and look at its structure
model = create_model()
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 120       
_________________________________________________________________
dropout (Dropout)            (None, 8)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 129
Trainable params: 129
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
# In the first case I will train the model on the full train1 dataset, with 80/20 Cross Validation split
training = model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=0)
# A metric looking at the accuracy of the fit
val_acc = np.mean(training.history['val_accuracy'])
print("\n%s: %.2f%%" % ('val_acc', val_acc*100))



Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set

In [21]:
# Now to apply a grid search to find best parameters for batch size and epoch.
# KerasClassifier is a wrapper that allows for k-fold cross validation as part of this grid search.
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the grid search parameters
batch_size = [16, 32, 64]
epochs = [50, 100, 150]
# Creating a grid in the form of a dictionary.
param_grid = dict(batch_size=batch_size, epochs=epochs)

# Search the grid
grid = GridSearchCV(estimator=model,
                   param_grid=param_grid, cv=3, verbose=10, n_jobs=1)

grid_result = grid.fit(X_train, Y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] batch_size=16, epochs=50 ........................................
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[CV] ............ batch_size=16, epochs=50, score=0.808, total=   1.4s
[CV] batch_size=16, epochs=50 ........................................
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this war

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[CV] ............ batch_size=16, epochs=50, score=0.828, total=   1.4s
[CV] batch_size=16, epochs=50 ........................................
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this war

...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[CV] ........... batch_size=32, epochs=100, score=0.825, total=   1.4s
[CV] batch_size=32, epochs=100 .......................................
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this war

[CV] ........... batch_size=32, epochs=150, score=0.855, total=   1.8s
[CV] batch_size=64, epochs=50 ........................................
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this war

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[CV] ........... batch_size=64, epochs=100, score=0.822, total=   1

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[CV] ........... batch_size=64, epochs=150, score=0.848, total=   1.2s
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_conv

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:   41.4s finished


In [22]:
# Summarise results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.829405 using {'batch_size': 32, 'epochs': 100}
0.827160 (0.015141) with: {'batch_size': 16, 'epochs': 50}
0.828283 (0.016495) with: {'batch_size': 16, 'epochs': 100}
0.829405 (0.019504) with: {'batch_size': 16, 'epochs': 150}
0.821549 (0.016722) with: {'batch_size': 32, 'epochs': 50}
0.829405 (0.014108) with: {'batch_size': 32, 'epochs': 100}
0.828283 (0.019824) with: {'batch_size': 32, 'epochs': 150}
0.809203 (0.013561) with: {'batch_size': 64, 'epochs': 50}
0.823793 (0.011111) with: {'batch_size': 64, 'epochs': 100}
0.828283 (0.014547) with: {'batch_size': 64, 'epochs': 150}


#### Note on outcome of first grid screening
It is interesting to note that the accuracy reported for the first pass of the model against the full train1 dataset with 80/20 split cross validation was 85.12%. However the optimal outcome of this grid screen above happens to be with the same epoch and batch size parameters that were used in the first pass, yet the accuracy here was only 82.94%.

The reason for this is likely to be either in how the cross validation breaks up the data, and indicates a level of instability for the model. Or, it relates to how the accuracy is calculated. I will return to this issue at the end of this project.

Interestingly I adjusted the seed from 42 to 23 (defined in create_model function) and the accuracy dropped to 84.95% so I do believe this has more to do with stochastic data sub-divisions than accuracy calculation methods.

In [23]:
# Optimisation algorithm
# Here I want to look at the range of available optimisation algorithms in Keras.
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=32, verbose=0)

# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Nadam']
param_grid = dict(opt=optimizer)

# search the grid
grid = GridSearchCV(estimator=model, param_grid=param_grid, verbose=2)
grid_result = grid.fit(X_train, Y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] opt=SGD .........................................................
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


/opt/anaconda3/envs/grace/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[CV] .......................................... opt=SGD, total=   1.3s
[CV] opt=SGD .........................................................
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this war

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s remaining:    0.0s


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[CV] .......................................... opt=SGD, total=   1.3s
[CV] opt=SGD .........................................................
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this war

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[CV] ...................................... opt=RMSprop, total=   1.4s
[CV] opt=Adagrad .....................................................
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this war

[CV] ..................................... opt=Adadelta, total=   1.3s
[CV] opt=Adadelta ....................................................
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this war

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[CV] ......................................... opt=Adam, total=   1

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:   26.2s finished


In [24]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.830528 using {'opt': 'Nadam'}
0.790123 (0.009655) with: {'opt': 'SGD'}
0.829405 (0.016109) with: {'opt': 'RMSprop'}
0.716049 (0.030157) with: {'opt': 'Adagrad'}
0.510662 (0.023381) with: {'opt': 'Adadelta'}
0.829405 (0.014108) with: {'opt': 'Adam'}
0.830528 (0.013837) with: {'opt': 'Nadam'}


In [25]:
# Hidden neurons
# This grid screen looks into different NN architechures. This includes looking at multiple hidden layers.
seed(42)
tf.random.set_seed(42)

# Create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=32, opt='Nadam', verbose=0)

# Define the grid search parameters
layers = [[7],[8],[10],[10,5],[12,6],[12,8,4]]

param_grid = dict(lyrs=layers)
# type(param_grid)
# Search the grid
grid = GridSearchCV(estimator=model, param_grid=param_grid, verbose=2)

grid_result = grid.fit(X_train, Y_train)


Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] lyrs=[7] ........................................................
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[CV] ......................................... lyrs=[7], total=   1.8s
[CV] lyrs=[7] ........................................................
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this war

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[CV] ......................................... lyrs=[7], total=   1.7s
[CV] lyrs=[7] ........................................................
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this war

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[CV] ......................................... lyrs=[8], total=   1.8s
[CV] lyrs=[10] .......................................................
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this war

[CV] ..................................... lyrs=[10, 5], total=   1.9s
[CV] lyrs=[10, 5] ....................................................
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this war

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[CV] ..................................... lyrs=[12, 6], total=   1

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:   32.7s finished


In [26]:
# Summarise results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.831650 using {'lyrs': [10]}
0.821549 (0.014547) with: {'lyrs': [7]}
0.830528 (0.013837) with: {'lyrs': [8]}
0.831650 (0.019244) with: {'lyrs': [10]}
0.821549 (0.014547) with: {'lyrs': [10, 5]}
0.821549 (0.017168) with: {'lyrs': [12, 6]}
0.829405 (0.009655) with: {'lyrs': [12, 8, 4]}


In [27]:
# Dropout refinement
# This refers to the potential for a node to be dropped out of the model.
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=32, opt='Nadam', verbose=0)

# Define the grid search parameters
drops = [0.0, 0.01, 0.05, 0.1, 0.2, 0.5]
param_grid = dict(dr=drops)
grid = GridSearchCV(estimator=model, param_grid=param_grid, verbose=2)
grid_result = grid.fit(X_train, Y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] dr=0.0 ..........................................................
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[CV] ........................................... dr=0.0, total=   1.7s
[CV] dr=0.0 ..........................................................
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this war

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.7s remaining:    0.0s


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[CV] ........................................... dr=0.0, total=   1.7s
[CV] dr=0.0 ..........................................................
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this war

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[CV] .......................................... dr=0.01, total=   2.0s
[CV] dr=0.05 .........................................................
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this war

[CV] ........................................... dr=0.1, total=   1.7s
[CV] dr=0.1 ..........................................................
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this war

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[CV] ........................................... dr=0.2, total=   1

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:   31.0s finished


In [28]:
# Summarise results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.831650 using {'dr': 0.01}
0.830528 (0.013837) with: {'dr': 0.0}
0.831650 (0.012598) with: {'dr': 0.01}
0.828283 (0.014547) with: {'dr': 0.05}
0.829405 (0.011446) with: {'dr': 0.1}
0.827160 (0.010408) with: {'dr': 0.2}
0.828283 (0.011983) with: {'dr': 0.5}


### Final Model

In [29]:
# Create final model
model = create_model(lyrs=[10], dr=0.01, opt='Nadam')

print(model.summary())

Model: "sequential_86"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_184 (Dense)            (None, 10)                150       
_________________________________________________________________
dropout_86 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_185 (Dense)            (None, 1)                 11        
Total params: 161
Trainable params: 161
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
# Train model on full train dataset with 80/20 CV split
training = model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=0)


# Evaluate the model
scores = model.evaluate(X_train, Y_train)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set

In [31]:
# As an exercise, reverting back to the original parameters that gave the 85.12% yields a lower score when compared 
# to the final model above. See below.
model1 = create_model(lyrs=[8], dr=0.0, opt='Adam')

print(model1.summary())


Model: "sequential_87"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_186 (Dense)            (None, 8)                 120       
_________________________________________________________________
dropout_87 (Dropout)         (None, 8)                 0         
_________________________________________________________________
dense_187 (Dense)            (None, 1)                 9         
Total params: 129
Trainable params: 129
Non-trainable params: 0
_________________________________________________________________
None


In [32]:

# Train model on full train dataset with 80/20 CV split
training = model1.fit(X_train, Y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=0)


# Evaluate the model
scores = model1.evaluate(X_train, Y_train)
print("\n%s: %.2f%%" % (model1.metrics_names[1], scores[1]*100))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set

## Application of final model to test1 data to generate a survival prediction

In [33]:
# Using the final model to calculate predictions for test dataset
#test1 = test1.drop(['Survived'], 1)
test1['Survived'] = model.predict(test1)
test1['Survived'] = test1['Survived'].apply(lambda x: round(x,0)).astype('int')
test2 = test1.copy()
test2['PassengerId'] = test['PassengerId']
solution = test2[['PassengerId', 'Survived']]
#test2
solution
#test1

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


### Output of a csv file that can be uploaded to Kaggle website for assessment

In [34]:
solution.to_csv("210526_Neural_Network_Solution1.csv", index=False)

## Results
Uploading this file into the kaggle website gave an accuracy for my model of 77.99%.
https://www.kaggle.com/c/titanic/submit

Reading online I discovered this drop in accuracy from train1 to test1 is not unusual as the test data has 'different patterns than the train data. 

https://www.kaggle.com/c/titanic/discussion/66780


## Discussion
Here is my first attempt at the creation and development of a Neural Network machine learning algorithm. I successfully trained my NN on the highly 'transformed' Titanic datasets and was able to get an accuracy for Survival prediction of 77.99%. 

Not a terrible outcome using the entry level NN package Keras. 